In [ ]:
# Programa alternativo con Timer y medida de jitter
# UMNG

from machine import ADC, Pin, Timer
import array, math, utime

# Configuración inicial
adc = ADC(Pin(27))     # Entrada ADC
N = 512                # Número de muestras
f_muestreo = 2000      # Frecuencia de muestreo (Hz)
dt_us = int(1_000_000 / f_muestreo)

muestras = array.array("H", [0] * N)
tiempos = array.array("f", [0.0] * N)

indice = 0
start_time = 0
fin_adquisicion = False

# Callback del Timer
def tomar_muestra(t):
    global indice, fin_adquisicion, start_time
    if indice == 0:
        start_time = utime.ticks_us()
    if indice < N:
        muestras[indice] = adc.read_u16()
        tiempos[indice] = utime.ticks_diff(utime.ticks_us(), start_time) / 1_000_000
        indice += 1
    else:
        fin_adquisicion = True
        t.deinit()

# Cálculo del jitter RMS
def calcular_jitter(tiempos, f_muestreo):
    Ts_ideal = 1.0 / f_muestreo
    intervalos = [tiempos[i] - tiempos[i-1] for i in range(1, len(tiempos))]
    errores = [(Ti - Ts_ideal) for Ti in intervalos]
    jitter_rms = math.sqrt(sum(e**2 for e in errores) / len(errores))
    return jitter_rms, intervalos

# Programa principal
def main():
    global fin_adquisicion, indice
    print("Iniciando adquisición estable con Timer...")

    timer = Timer()
    timer.init(period=dt_us, mode=Timer.PERIODIC, callback=tomar_muestra)

    # Esperar hasta completar adquisición
    while not fin_adquisicion:
        pass

    elapsed = tiempos[-1]
    fs_real = N / elapsed
    print(f"Frecuencia deseada: {f_muestreo} Hz, frecuencia real: {fs_real:.2f} Hz")

    # Calcular jitter
    jitter_rms, intervalos = calcular_jitter(tiempos, f_muestreo)
    print(f"Jitter RMS: {jitter_rms*1e6:.2f} us")

    # Guardar datos en archivo
    with open("muestras_timer.txt", "w") as f:
        f.write("Tiempo(s)\tVoltaje(V)\n")
        voltajes = [(x / 65535) * 3.3 for x in muestras]
        for t, v in zip(tiempos, voltajes):
            f.write(f"{t:.6f}\t{v:.5f}\n")

    with open("jitter.txt", "w") as f:
        f.write("Intervalo(s)\n")
        for Ti in intervalos:
            f.write(f"{Ti:.8f}\n")

    print("Adquisición completada, datos y jitter guardados.")

if __name__ == "__main__":
    main()